In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import datetime
import scipy.stats as stats

#graphing
import matplotlib.pyplot as plt
#stats
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel
from sklearn.mixture import GaussianMixture 
#import testing
import sys
sys.path.append("../")
import selection_tests

In [2]:
data = pd.read_stata("all_plans_c.dta")
data['log_enroll'] = np.log(data['enr_c'])
data['enr_total'] = data['enr_c'] + data['enr_FFS']
data = data[data['year'] > 2011]
print(data.columns)

Index(['ssa', 'state', 'county', 'year', 'enr_FFS', 'enr_c', 'hhi_ins',
       'hhi_ins_noSNP', 'ins_parent', 'ins_parent_noSNPs', 'ins_plans',
       'HMO_share', 'PPO_share', 'qual_2012', 'qual_2013', 'qual_2014',
       'qual_2015', 'partaenrollment', 'partb_enrollment',
       'prescription_drugs', 'prev_comp_dental', 'eye_exams', 'hearing_exams',
       'deductible', 'partb_premium', 'plan_premium', 'partd_premium', 'OOPC',
       'risk_pub_p', 'bid_pub_p', 'rebate_pub_p', 'risk_pub_c', 'bid_pub_c',
       'rebate_pub_c', 'star_C2', 'star_CD2', 'bmFFS', 'bm_ns', 'risk_FFS',
       'FFS_AB', 'FFS_AB_rs', 'buydown', 'OOPC_noprem', 'extras', 'quartile',
       'bid_pub_p_nominal', 'bid_pub_c_nominal', 'rebate_pub_p_nominal',
       'rebate_pub_c_nominal', 'bmFFS_nominal', 'FFS_AB_nominal',
       'bm_ns_nominal', 'OOPC_nominal', 'plan_premium_nominal',
       'partd_premium_nominal', 'buydown_nominal', 'rebate_std',
       'benchmark_diff', 'benchmark_diff_n', 'benchmark_diff_ns', 'q

In [3]:
#https://www.statsmodels.org/dev/examples/notebooks/generated/glm_weights.html

def drop_data(data,y_name,x_name,absorb=None):
    data = data.copy()
    data = data[y_name + x_name + absorb]
    missing_vals = ~data.isnull().max(axis=1)
    data = data[missing_vals]
    data = data[data['year'].groupby(data['ssa']).transform('count')>=11]
    return data
    

def demean(y_name,x_name,data=None,absorb=None,cluster=None): 

    y,X = data[ y_name], data[ x_name ]
    
    y_dot = y.copy()
    X_dot = X.copy()
    
    ybar = y.mean()
    Xbar = X.mean()

    
    for effect in absorb:
        y_dot = y_dot - y.groupby(data[effect]).transform('mean')
        X_dot = X_dot - X.groupby(data[effect]).transform('mean') 
    
    y_dot = y_dot + ybar
    X_dot = X_dot + Xbar
    return y_dot, X_dot

In [4]:
class GLS_LL(GenericLikelihoodModel):

    def __init__(self, *args, model=None, **kwargs):
        super(GLS_LL, self).__init__(*args, **kwargs)
        self.model = model
        
    def loglikeobs(self, params, scale=None):
        """
        Evaluate the log-likelihood for a generalized linear model.
        """
        model = self.model
        scale = sm.tsa.stattools.float_like(scale, "scale", optional=True)
        lin_pred = np.dot(model.exog, params) + model._offset_exposure
        expval = model.family.link.inverse(lin_pred)
        if scale is None:
            scale = model.estimate_scale(expval)
        llf = model.family.loglike_obs(model.endog, expval, model.var_weights,
                                  scale)
        return llf

    
def print_results(y_name,data,weights=True):
    
    x_name = ['bmFFS', 'log_risk_pub_c','FFS_AB',"ins_parent_noSNPs",'unemploy_rt',
              'pc_income','rebate_pub_c']
    #'post_bmFFS',
    y_dot, X_dot = demean(y_name,x_name, data=data,absorb=['ssa','year'])
    missing_vals = ~data[y_name + x_name].isnull().max(axis=1)
    y_dot, X_dot = y_dot[missing_vals],X_dot[missing_vals]
    gmmodel = GaussianMixture (n_components=2)

    #model 1 set up data
    data_stack = X_dot[['log_risk_pub_c']].copy()
    data_stack['y'] = y_dot
    classify = GaussianMixture (n_components=2).fit(data_stack)
    c = np.array(classify.predict(data_stack))

    classify = gmmodel.fit(data_stack)
    c = np.array(classify.predict(data_stack))
    
    print(c.sum()/c.shape[0] )
    print(data[missing_vals][x_name+['risk_pub_c']][c==0].mean() )
    print(data[missing_vals][x_name+['risk_pub_c']][c==1].mean() )
    
    #model 2 set up data
    X_dot2 = X_dot.copy()
    X_dot2['bmFFS*c'] = X_dot['bmFFS']*c
    

    m1,m2 = None,None
    if weights:
        weights = np.array( data['enr_c'][missing_vals] )
        m1 = sm.GLM(y_dot,X_dot,var_weights=weights)
        m2 = sm.GLM(y_dot,X_dot2,var_weights=weights)
    else:
        m1 = sm.GLM(y_dot,X_dot)
        m2 = sm.GLM(y_dot,X_dot2)
        
    #model 1
    m1_fit = m1.fit()
    print(m1_fit.summary().as_latex())
    m2_fit = m2.fit()
    print(m2_fit.summary().as_latex())
    
print_results(['log_enroll'],data)

0.2502170408036711
bmFFS                806.836243
log_risk_pub_c        -6.476725
FFS_AB               697.544678
ins_parent_noSNPs      2.891324
unemploy_rt            6.205707
pc_income             39.985490
rebate_pub_c          45.108919
risk_pub_c             0.943594
dtype: float64
bmFFS                804.860901
log_risk_pub_c        -8.548820
FFS_AB               721.982910
ins_parent_noSNPs      1.786617
unemploy_rt            6.563941
pc_income             38.069699
rebate_pub_c          30.189500
risk_pub_c             0.930363
dtype: float64
\begin{center}
\begin{tabular}{lclc}
\toprule
\textbf{Dep. Variable:}      &   log\_enroll    & \textbf{  No. Observations:  } &    16126    \\
\textbf{Model:}              &       GLM        & \textbf{  Df Residuals:      } &    16119    \\
\textbf{Model Family:}       &     Gaussian     & \textbf{  Df Model:          } &        6    \\
\textbf{Link Function:}      &     identity     & \textbf{  Scale:             } &    46.290   \\
\

In [5]:
#this is janky as f.... need to fix it...
def setup_test(yn, xn,
    y_name = ['log_enroll'],
    x_name = ['bmFFS','log_risk_pub_c','FFS_AB',"ins_parent_noSNPs",'log_risk_FFS','unemploy_rt','pc_income']):
    
    data = xn.copy()

    #data1 = drop_data(data,y_name,x_name,absorb=['ssa','year'])
    y_dot, X_dot = demean(y_name,x_name, data=data,absorb=['ssa','year'])
    missing_vals = ~data[y_name + x_name].isnull().max(axis=1)
    y_dot, X_dot = y_dot[missing_vals],X_dot[missing_vals]

    #double check if weighting + claissifer helps
    gmmodel = GaussianMixture (n_components=2)

    #high risk score, low enrollment? much bigger benchmark effect...
    data_stack = X_dot[['log_risk_pub_c']].copy()
    data_stack['y'] = y_dot
    classify = GaussianMixture (n_components=2).fit(data_stack)
    c = np.array(classify.predict(data_stack))

    classify = gmmodel.fit(data_stack)
    c = np.array(classify.predict(data_stack))

    #model 1
    weights = np.array( data['enr_c_mean'][missing_vals] )
    m1 = sm.GLM(y_dot,X_dot)#,var_weights=weights)
    m1_fit = m1.fit()
    

    #model2
    X_dot2 = X_dot.copy()
    X_dot2['bmFFS*c'] = X_dot['bmFFS']*c
    m2 = sm.GLM(y_dot,X_dot2)#,var_weights=weights)
    m2_fit = m2.fit()

    model1 = GLS_LL(y_dot, X_dot, model=m1)
    ll1 = model1.loglikeobs(m1_fit.params)
    grad1 = model1.score_obs(m1_fit.params)
    hess1 = model1.hessian(m1_fit.params)
    params1 = m1_fit.params

    # fit logistic values
    model2 = GLS_LL(y_dot, X_dot, model=m2)
    ll2 = model2.loglikeobs(m2_fit.params)
    grad2 = model2.score_obs(m2_fit.params)
    hess2 = model2.hessian(m2_fit.params)
    params2 = m2_fit.params

    return  ll1, grad1, hess1, params1, ll2, grad2, hess2, params2


print(selection_tests.test_table(data['log_enroll'],data,setup_test))

\begin{center}
\begin{tabular}{ccccc}
\toprule
\textbf{Version} & \textbf{Result} & \textbf{90 \% CI} & \textbf{95 \% CI} & \textbf{99 \% CI} \\ \midrule
Shi (2015) & H0 & [-6.200, 2.179] & [-9.510, 5.494] & [-15.911, 11.890] \\
Classical & H2 & [-3.782, -0.492] & [-4.096, -0.178] & [-4.713, 0.439] \\
Bootstrap & H0 & [-3.788, -0.298] & [-4.219, 0.066] & [-4.829, 0.571] \\
\bottomrule
\end{tabular}
\end{center}
None
